# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=150, n_features=4, n_informative=4, n_redundant=0, n_clusters_per_class=1, flip_y=0.2, class_sep=0.05)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 52,  39,  37,  50, 108,   5,  61,  37,  53,  73,  33,  32, 105,
        27,   0,  44,  25, 105,  78,  14,  22,  29,  21,  75,  12, 105,
        90, 111,  17,  73, 108, 106,  36,  69,  59,   3,  76,  90,  59,
        33,  54,  82,  87, 105,  41,  15,  27,  37,  17,  46,  84,  35,
        33,  88,  69,  52, 101,  73,   7,  39,  71,  56,  99,  44,  94,
       108,  72,   3,  51,  57,  47,   8,   4,  97,  15,  89,  46,  52,
        25,  98,  23,  43,  14,  67,  92,  18,  90,  88,  10,  62,  76,
       101,  58,  16,  50,  87,  34, 110,   1,  52,  69,  63,  21,  77,
       109,  57,  34,  59,   4,  37, 108,  91])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({50: 1,
         60: 2,
         107: 1,
         91: 1,
         110: 2,
         104: 3,
         46: 1,
         28: 1,
         34: 2,
         15: 2,
         7: 1,
         52: 2,
         0: 1,
         73: 1,
         22: 2,
         24: 3,
         80: 1,
         27: 3,
         62: 1,
         90: 1,
         36: 1,
         25: 1,
         51: 1,
         37: 1,
         82: 2,
         13: 3,
         61: 1,
         58: 1,
         14: 1,
         69: 3,
         106: 1,
         23: 4,
         86: 2,
         31: 3,
         48: 1,
         75: 1,
         2: 1,
         66: 1,
         97: 2,
         20: 1,
         89: 1,
         95: 1,
         17: 2,
         30: 2,
         94: 2,
         18: 2,
         1: 1,
         83: 2,
         111: 1,
         72: 4,
         92: 3,
         56: 1,
         21: 1,
         70: 1,
         35: 1,
         54: 1,
         105: 1,
         32: 2,
         53: 1,
         63: 1,
         12: 3,
         43: 1,
      

In [6]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [7]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [8]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({2: 38, 0: 37, 1: 37})
Classes distribution in the bootstrap: Counter({2: 40, 1: 40, 0: 32})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [12]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9736842105263158


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [16]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=3), n_estimators=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

1.0


## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [19]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    indices = np.random.choice(
        np.arange(X.shape[1]), size=int(np.sqrt(X.shape[1])), replace=False
    )
    X_train_i = X_train_i[:,indices]
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test[:,indices]))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9736842105263158


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.9736842105263158


In [ ]:
from figures import plot_forest_interactive
plot_forest_interactive()